In [27]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
import joblib as jl
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [28]:
df="CAR_DETAILS.csv"
df=pd.read_csv(df)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           4340 non-null   object
 1   year           4340 non-null   int64 
 2   selling_price  4340 non-null   int64 
 3   km_driven      4340 non-null   int64 
 4   fuel           4340 non-null   object
 5   seller_type    4340 non-null   object
 6   transmission   4340 non-null   object
 7   owner          4340 non-null   object
dtypes: int64(3), object(5)
memory usage: 271.4+ KB
None


In [29]:
print("Null values in dataset: ",df.isnull().count().sum())
print("Duplicate values in dataset: ",df.duplicated().sum())
df1=df.drop_duplicates()
df1=df1.drop('name',axis=1)
print("Duplicate values in dataset: ",df1.duplicated().sum())

Null values in dataset:  34720
Duplicate values in dataset:  763
Duplicate values in dataset:  79


In [30]:
num=["km_driven","year",'selling_price']
cate=["seller_type","owner","fuel","transmission"]

scale=StandardScaler()
scaled=scale.fit_transform(df1[num])

le = LabelEncoder()
for col in cate:
    df1[col] = le.fit_transform(df1[col])

print(df1.head())

   year  selling_price  km_driven  fuel  seller_type  transmission  owner
0  2007          60000      70000     4            1             1      0
1  2007         135000      50000     4            1             1      0
2  2012         600000     100000     1            1             1      0
3  2017         250000      46000     4            1             1      0
4  2014         450000     141000     1            1             1      2


In [31]:
x_s=df1.drop('selling_price',axis=1)
y = df1['selling_price']
col_names=x_s.columns
x=pd.DataFrame(x_s,columns=col_names)
print(df1.columns)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

Index(['year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner'],
      dtype='object')


In [32]:
def reg_eval(y,ypred):
    print('MAE',mean_absolute_error(y,ypred))
    print('MSE',mean_squared_error(y,ypred))
    print('RMSE',np.sqrt(mean_squared_error(y,ypred)))
    print('R2 score',r2_score(y,ypred))
    print('\n')

def model_score(model):
  print('Training Score',model.score(x_train,y_train))
  print('Testing Score',model.score(x_test,y_test))

In [33]:
model_lin=LinearRegression()
model_lin.fit(x_train,y_train)

ypred1=model_lin.predict(x_test)

print('Evalutaing the Model Linear Regression\n')
reg_eval(y_test,ypred1)
model_score(model_lin)

Evalutaing the Model Linear Regression

MAE 215522.4645487041
MSE 197896678171.88495
RMSE 444855.7948053335
R2 score 0.38566846491225903


Training Score 0.4405812131213911
Testing Score 0.38566846491225903


In [34]:
model_knn=KNeighborsRegressor(n_neighbors=11)
model_knn.fit(x_train,y_train)

ypred2=model_knn.predict(x_test)

print('Evalutaing the Model: KNN\n')
reg_eval(y_test,ypred2)
model_score(model_knn)

Evalutaing the Model: KNN

MAE 239459.16861350942
MSE 276148439805.9148
RMSE 525498.2776431477
R2 score 0.14275117447548002


Training Score 0.3134740452692356
Testing Score 0.14275117447548002


In [35]:
model_ridge=Ridge(alpha=0.5)
model_ridge.fit(x_train,y_train)
ypred4=model_ridge.predict(x_test)

print('Evalutaing the Model Ridge\n')
reg_eval(y_test,ypred4)
model_score(model_ridge)

Evalutaing the Model Ridge

MAE 215471.1465997805
MSE 197889561470.12796
RMSE 444847.7958472178
R2 score 0.38569055732106305


Training Score 0.44058019228198664
Testing Score 0.38569055732106305


In [36]:
pickle.dump(model_knn,open("Car_price_predict.pkl",'wb'))

loaded=pickle.load(open('Car_price_predict.pkl','rb'))

In [37]:
sample= df1.sample(20,random_state=42)

print("Training Features:", x.shape[1])
print("Sample Features:", df1.shape[1])
print(x.columns)



sample_predictions = loaded.predict(x)
print("Predicted Prices:", sample_predictions)

Training Features: 6
Sample Features: 7
Index(['year', 'km_driven', 'fuel', 'seller_type', 'transmission', 'owner'], dtype='object')
Predicted Prices: [120090.90909091 135454.54545455 360999.90909091 ... 359181.81818182
 583454.45454545 314090.81818182]


In [38]:
prediction= loaded.predict(x)
print("Prediction is : ",prediction)

Prediction is :  [120090.90909091 135454.54545455 360999.90909091 ... 359181.81818182
 583454.45454545 314090.81818182]
